In [3]:
import numpy as np
import os
from glob import glob
from tqdm import tqdm
from scipy.signal import medfilt
import cv2

In [4]:
frame_path = r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\unzipped_video" # <-- Set this to your frame directory
output_path = "mesh_stabilized.mp4"
mesh_size = 16  # Mesh cell size in pixels (adjust as needed)
feature_max = 400  # Max features per frame
radius = 100  # Motion propagation radius (pixels)
window_size = 15  # Smoothing window (frames)
crop_margin = 30  # Cropping margin to remove border artifacts

# --- LOAD FRAMES ---
frame_files = sorted(glob(os.path.join(frame_path, "*.jpg")))
frames = [cv2.imread(f) for f in frame_files]
height, width = frames[0].shape[:2]

# --- VIDEO WRITER ---
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (width - 2*crop_margin, height - 2*crop_margin))

# --- MESH GRID ---
rows = height // mesh_size
cols = width // mesh_size

def get_mesh_vertices():
    yv, xv = np.meshgrid(np.arange(0, height, mesh_size), np.arange(0, width, mesh_size), indexing='ij')
    return np.stack([xv, yv], axis=-1).reshape(-1, 2)

mesh_vertices = get_mesh_vertices()

# --- FEATURE DETECTOR ---
def detect_features(gray):
    return cv2.goodFeaturesToTrack(gray, maxCorners=feature_max, qualityLevel=0.01, minDistance=8)

# --- MOTION PROPAGATION ---
def mesh_motion(old_pts, new_pts, old_gray):
    # Assign motion vectors to mesh vertices using median filtering
    motion_vectors = np.zeros((rows, cols, 2), dtype=np.float32)
    counts = np.zeros((rows, cols), dtype=np.int32)
    for (p0, p1) in zip(old_pts, new_pts):
        x, y = p0.ravel()
        dx, dy = (p1 - p0).ravel()
        i, j = int(y) // mesh_size, int(x) // mesh_size
        if 0 <= i < rows and 0 <= j < cols:
            motion_vectors[i, j] += [dx, dy]
            counts[i, j] += 1
    # Avoid division by zero
    counts[counts == 0] = 1
    motion_vectors /= counts[..., None]
    # Median filter to suppress outliers (moving objects)
    motion_vectors[...,0] = medfilt(motion_vectors[...,0], kernel_size=3)
    motion_vectors[...,1] = medfilt(motion_vectors[...,1], kernel_size=3)
    return motion_vectors

# --- PATH SMOOTHING ---
def smooth_path(path, radius=window_size):
    smoothed = np.copy(path)
    for i in range(path.shape[0]):
        for j in range(path.shape[1]):
            for d in range(2):
                smoothed[i,j,:,d] = np.convolve(path[i,j,:,d], np.ones(radius)/radius, mode='same')
    return smoothed

# --- WARP FRAME WITH MESH ---
def warp_frame_mesh(frame, mesh_flow):
    map_x = np.zeros((height, width), dtype=np.float32)
    map_y = np.zeros((height, width), dtype=np.float32)
    for i in range(rows):
        for j in range(cols):
            dx, dy = mesh_flow[i, j]
            y0 = i * mesh_size
            x0 = j * mesh_size
            y1 = min((i+1) * mesh_size, height)
            x1 = min((j+1) * mesh_size, width)
            map_x[y0:y1, x0:x1] = np.clip(np.arange(x0, x1)[None, :] + dx, 0, width-1)
            map_y[y0:y1, x0:x1] = np.clip(np.arange(y0, y1)[:, None] + dy, 0, height-1)
    return cv2.remap(frame, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

# --- MAIN PROCESS ---
prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
prev_pts = detect_features(prev_gray)
mesh_flows = []

# Step 1: Estimate mesh flows between frames
for i in tqdm(range(1, len(frames)), desc="Estimating mesh flows"):
    curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
    curr_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)
    good_prev = prev_pts[status.flatten()==1]
    good_curr = curr_pts[status.flatten()==1]
    mesh_flow = mesh_motion(good_prev, good_curr, prev_gray)
    mesh_flows.append(mesh_flow)
    prev_gray = curr_gray
    prev_pts = detect_features(curr_gray)

# Step 2: Accumulate and smooth mesh flows
mesh_flows = [np.zeros_like(mesh_flows[0])] + mesh_flows  # First frame has zero motion
mesh_paths = np.cumsum(mesh_flows, axis=0)  # Accumulate
mesh_paths = np.stack(mesh_paths, axis=2)  # Shape: (rows, cols, frames, 2)
mesh_paths = np.transpose(mesh_paths, (0,1,2,3))  # Ensure correct shape
mesh_paths = smooth_path(mesh_paths)

# Step 3: Warp and save stabilized frames
for i, frame in tqdm(enumerate(frames), total=len(frames), desc="Warping and saving"):
    mesh_flow = mesh_paths[...,i,:]
    stabilized = warp_frame_mesh(frame, mesh_flow)
    # Crop borders to remove artifacts
    stabilized = stabilized[crop_margin:height-crop_margin, crop_margin:width-crop_margin]
    out.write(stabilized)

out.release()
print("Stabilized video saved to", output_path)

Estimating mesh flows:   0%|          | 4/7168 [00:01<48:14,  2.47it/s]


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 11059200 bytes in function 'cv::OutOfMemoryError'
